In [85]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage
import math
import hashlib

def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_colab import *


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# These will already be installed in the testing environment so disregard the
# amount of time (~1 minute) it takes to install.
!pip install -q pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 121749 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack ..

In [3]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
conf.set("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

In [4]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
import signal

AUTH_TIMEOUT = 30000000

def handler(signum, frame):
  raise Exception("Authentication timeout!")

signal.signal(signal.SIGALRM, handler)
signal.alarm(AUTH_TIMEOUT)

try:
   auth.authenticate_user()
except:
   pass

In [5]:
# Copy one wikidumps files
import os
from pathlib import Path
from google.colab import auth
## RENAME the project_id to yours project id from the project you created in GCP
project_id = 'informationretrievalex3'
!gcloud config set project {project_id}

data_bucket_name = 'wikidata20210801_preprocessed'
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass
except:
      !mkdir wikidumps
      !gsutil -u {project_id} cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/"

Updated property [core/project].
Copying gs://wikidata20210801_preprocessed/multistream1_preprocessed.parquet...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

/ [1 files][316.7 MiB/316.7 MiB]                                                
Operation completed over 1 objects/316.7 MiB.                                    


In [6]:
from pathlib import Path
import os

try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

parquetFile = spark.read.parquet(path)
# take the 'text' and 'id' or the first 1000 rows and create an RDD from it
doc_text_pairs = parquetFile.limit(1000).select("text", "id").rdd


[Row(text='\'\'\'Anarchism\'\'\' is a political philosophy and movement that is sceptical of authority and rejects all involuntary, coercive forms of hierarchy. Anarchism calls for the abolition of the state, which it holds to be undesirable, unnecessary, and harmful. As a historically far-left movement, it is usually described alongside libertarian Marxism as the libertarian wing (libertarian socialism) of the socialist movement and has a strong historical association with anti-capitalism and socialism.\n\nThe history of anarchy goes back to prehistory, when humans arguably lived in anarchic societies long before the establishment of formal states, realms or empires. With the rise of organised hierarchical bodies, scepticism toward authority also rose, but it was not until the 19th century that a self-conscious political movement emerged. During the latter half of the 19th and the first decades of the 20th century, the anarchist movement flourished in most parts of the world and had a

**Count the frequency of each word in docunment**

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)

"""
Count the frequency of each word in docunment
Parameters:
  text: (str) Text of one document
  id: (int) Document id
  withTFnormal (bool) if normalization by leanght of the document
Returns:
  List of tuples: A list of (token, (doc_id, tf))
"""

l = ["barcelona", "ring"]
def word_count(text, id, withTFnormal=False, keyIsDoc=False):
  containl = False
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  sizeOfDoc = 0
  word_counts = Counter()

  for token in tokens:
    if (token not in all_stopwords):
      word_counts[token] += 1
      sizeOfDoc += 1
      if(containl == False):
          if(token in l):
            containl = True

  if(containl == False):
    return []

  if (keyIsDoc):
    return [(id, (token, count/sizeOfDoc)) for token, count in word_counts.items()]

  if(not withTFnormal):
    return [(token, (id, count)) for token, count in word_counts.items()]

  return [(token, (id, count/sizeOfDoc)) for token, count in word_counts.items()]



In [ ]:
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))    # Count the frequency of each word in docunment without normalization

**sorted posting list by wiki_id**

In [14]:
"""
Returns a sorted posting list by wiki_id.
Parameters:
  unsorted_pl: list of tuples - A list of (wiki_id, tf) tuples
Returns:
    list of tuples A sorted posting list.
"""
def reduce_word_counts(unsorted_pl):
  return sorted(unsorted_pl, key=lambda x: x[0])

In [ ]:
postings = word_counts.groupByKey().mapValues(reduce_word_counts)   # Sort by document id and then merge by term.
postings_filtered = postings.filter(lambda x: len(x[1])>10)         # Filter out rare words

**Takes a posting list RDD and calculate the df for each token**

In [ ]:
"""
Takes a posting list RDD and calculate the df for each token.
Parameters:
  postings: (RDD) An RDD where each element is a (token, posting_list).
Returns:
  An RDD where each element is a (token, df) pair.
"""
def calculate_df(postings):
  return postings.map(lambda x: (x[0], len(x[1])))

In [ ]:
w2df_dict = (calculate_df(postings_filtered)).collectAsMap()    # Create dict of df

**A function that partitions the posting lists into buckets, writes out all posting lists in a bucket to disk**

In [ ]:
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

"""
A function that partitions the posting lists into buckets, writes out all posting lists in a bucket to disk.
Parameters:
  postings: (RDD) An RDD where each item is a (w, posting_list) pair.
Returns:
  An RDD where each item is a posting locations dictionary for a bucket.
"""
def partition_postings_and_write(postings):

  bucketed_postings = postings.map(lambda x: (token2bucket_id(x[0]), [(x[0], x[1])]))         # Assign each token to a bucket
  grouped_postings = bucketed_postings.reduceByKey(lambda x, y: x + y)                        # Group the postings by bucket
  posting_locations = grouped_postings.map(lambda x: InvertedIndex.write_a_posting_list(x))   # Write out each bucket and collect posting locations

  return posting_locations

In [ ]:
# Merge the posting locations into a single dict.
posting_locs_list = partition_postings_and_write(postings).collect()
super_posting_locs = defaultdict(list)
for posting_loc in posting_locs_list:
  for k, v in posting_loc.items():
    super_posting_locs[k].extend(v)


**Create tf-idf dictionary**

In [ ]:
# Create tf-idf dictionary

N = 1000;

word_TF = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1], True))
postings_TF = word_TF.groupByKey().mapValues(reduce_word_counts)
postings_TF_filtered = postings_TF.filter(lambda x: len(x[1])>10)

postings_TF_IDF_filtered = postings_TF_filtered.map(lambda x: (x[0], updated_value_function(x[0],x[1])))

def updated_value_function(term, post):
    return list(map(lambda x: (x[0], x[1]*math.log10(N/w2df_dict[term])), post))

postings_TF_IDF_filtered_dict = postings_TF_IDF_filtered.collectAsMap()

**Create nf dictionary**

In [ ]:
Doc_TF = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1], True,True))
Doc_TF = Doc_TF.groupByKey().mapValues(reduce_word_counts)

Doc_TF_IDF_filtered = Doc_TF.map(lambda x: (x[0], updated_value_function2(x[1])))
def updated_value_function2(post):
  listTermTfIdF = []

  for term, tf in post:
    if (term in w2df_dict):
      listTermTfIdF.append((term, tf * math.log10(N / w2df_dict[term])))

  nf = 0
  for term, tfIdf in listTermTfIdF:
    nf = nf + tfIdf ** 2
  nf = 1/math.sqrt(nf)

  return nf

Doc_TF_IDF_filtered_dict = Doc_TF_IDF_filtered.collectAsMap()

**write the global stats out**

In [ ]:
inverted = InvertedIndex()                            # Create inverted index instance
inverted.posting_locs = super_posting_locs            # Adding the posting locations dictionary to the inverted index
inverted.df = w2df_dict                               # Add the df dictionary to the inverted index
inverted.tfidf = postings_TF_IDF_filtered_dict        # Add the tfidf dictionary to the inverted index
inverted.nf = Doc_TF_IDF_filtered_dict
inverted.write_index('.', 'index')                    # write the global stats out


In [ ]:
inverted_index = InvertedIndex.read_index('.', 'index')

**Title**

In [68]:
parquetFile = spark.read.parquet(path)
# take the 'text' and 'id' or the first 1000 rows and create an RDD from it
doc_text_pairs = parquetFile.limit(1000).select("title", "id").rdd


In [69]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)

def word_title_count(title, id):
    tokens = [token.group() for token in RE_WORD.finditer(title.lower())]
    sizeOfTitle = 0
    word_counts = Counter()

    for token in tokens:
        if (token not in all_stopwords):
            word_counts[token] += 1
            sizeOfTitle += 1

    return [(token, (id, count/sizeOfTitle)) for token, count in word_counts.items()]

In [70]:
def reduce_word_title_counts(unsorted_pl):
  return sorted(unsorted_pl, key=lambda x: x[0])

In [84]:
word_counts = doc_text_pairs.flatMap(lambda x: word_title_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_title_counts)   # Sort by document id and then merge by term.
postings_filtered = postings.filter(lambda x: len(x[1])>10)
postings_filtered = postings_filtered.collect()
print(postings_filtered)

[('american', [(309, 0.5), (615, 0.3333333333333333), (659, 0.25), (771, 0.3333333333333333), (863, 0.3333333333333333), (869, 0.3333333333333333), (966, 0.5), (1110, 0.3333333333333333), (1111, 0.3333333333333333), (1112, 0.3333333333333333), (1201, 0.3333333333333333), (1241, 0.5), (1488, 0.5), (1558, 0.3333333333333333), (1890, 0.5), (1902, 0.3333333333333333), (1950, 0.25), (1973, 0.5), (2011, 0.25), (2175, 0.5), (2357, 0.3333333333333333), (2386, 0.5)]), ('disambiguation', [(590, 0.5), (632, 0.5), (661, 0.5), (679, 0.5), (694, 0.3333333333333333), (780, 0.5), (1155, 0.5), (1184, 0.5), (1391, 0.5), (1501, 0.5), (1597, 0.3333333333333333), (1919, 0.5), (1931, 0.5), (2004, 0.5), (2123, 0.5), (2127, 0.5), (2209, 0.5), (2332, 1.0)]), ('apollo', [(594, 1.0), (662, 1.0), (663, 1.0), (1461, 0.5), (1770, 1.0), (1773, 1.0), (1774, 1.0), (1965, 1.0), (1966, 1.0), (1967, 1.0), (1968, 1.0), (1969, 1.0), (1970, 1.0), (1971, 1.0)]), ('albert', [(711, 0.3333333333333333), (736, 0.5), (954, 0.5), 

In [86]:
inverted = InvertedIndex()                            # Create inverted index instance
inverted.tfTitle = postings_filtered
inverted.write_index('.', 'index')

In [89]:
inverted_index = InvertedIndex.read_index('.', 'index')
print(inverted_index.tfTitle)

[('american', [(309, 0.5), (615, 0.3333333333333333), (659, 0.25), (771, 0.3333333333333333), (863, 0.3333333333333333), (869, 0.3333333333333333), (966, 0.5), (1110, 0.3333333333333333), (1111, 0.3333333333333333), (1112, 0.3333333333333333), (1201, 0.3333333333333333), (1241, 0.5), (1488, 0.5), (1558, 0.3333333333333333), (1890, 0.5), (1902, 0.3333333333333333), (1950, 0.25), (1973, 0.5), (2011, 0.25), (2175, 0.5), (2357, 0.3333333333333333), (2386, 0.5)]), ('disambiguation', [(590, 0.5), (632, 0.5), (661, 0.5), (679, 0.5), (694, 0.3333333333333333), (780, 0.5), (1155, 0.5), (1184, 0.5), (1391, 0.5), (1501, 0.5), (1597, 0.3333333333333333), (1919, 0.5), (1931, 0.5), (2004, 0.5), (2123, 0.5), (2127, 0.5), (2209, 0.5), (2332, 1.0)]), ('apollo', [(594, 1.0), (662, 1.0), (663, 1.0), (1461, 0.5), (1770, 1.0), (1773, 1.0), (1774, 1.0), (1965, 1.0), (1966, 1.0), (1967, 1.0), (1968, 1.0), (1969, 1.0), (1970, 1.0), (1971, 1.0)]), ('albert', [(711, 0.3333333333333333), (736, 0.5), (954, 0.5), 

In [ ]:
simDoc = Counter()
query = "ring"
query = dict(word_count(query, 0, True))
for term, value in query.items():
  query[term] = (value[0], value[1] * math.log10(N / inverted.df[term]))

nfQuery = 0
for term, value in query.items():
  nfQuery = nfQuery + value[1] ** 2
nfQuery = 1/math.sqrt(nfQuery)


for term,  value in query.items():
  docTfItf = inverted.tfidf[term]
  for doc, weight in docTfItf:
    simDoc[doc] += value[1] * weight

for doc, sim in simDoc.items():
  simDoc[doc] = simDoc[doc] * nfQuery * inverted.nf[doc]

top_10_similar_docs = simDoc.most_common(10)

print(top_10_similar_docs)

[(316, 1.0), (639, 1.0), (657, 1.0), (666, 1.0), (674, 1.0), (679, 1.0), (680, 1.0), (737, 1.0), (740, 1.0), (771, 1.0)]
